<a href="https://colab.research.google.com/github/gracialukelo/fetch_dynamic_data/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# main.py

# Installationsbefehle (entferne die Kommentarzeichen, wenn du sie ausführen möchtest)
# pip install nest_asyncio playwright beautifulsoup4 fpdf google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

import asyncio
import logging
from pathlib import Path
from web_scraper import WebScraper  # Stelle sicher, dass web_scraper.py im gleichen Verzeichnis liegt
from google_drive_uploader import GoogleDriveUploader  # Stelle sicher, dass google_drive_uploader.py im gleichen Verzeichnis liegt

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

async def main():
    # WebScraper konfigurieren
    url = "https://sites.google.com/roche.com/site-mannheim-area-mgmt/site-area-management"  # Ersetze dies durch die gewünschte URL
    output_file = "extracted_teaser_data"
    output_format = "pdf"  # Kann "txt", "pdf" oder "json" sein
    output_dir = "data"  # Ausgabeordner

    scraper = WebScraper(
        url=url,
        output_file=output_file,
        output_format=output_format,
        output_dir=output_dir
    )

    # Scraping-Prozess starten
    await scraper.run()

    # GoogleDriveUploader konfigurieren
    SERVICE_ACCOUNT_FILE = 'service_account.json'
    PARENT_FOLDER_ID = "19l__xrAF6V9OkC0q327uh1Y0XARh1y8E"  # Google Drive Ordner ID

    uploader = GoogleDriveUploader(
        service_account_file=SERVICE_ACCOUNT_FILE,
        parent_folder_id=PARENT_FOLDER_ID
    )

    # Authentifizierung durchführen
    try:
        uploader.authenticate()
    except Exception:
        logging.critical("Programm wird beendet aufgrund eines Authentifizierungsfehlers.")
        return

    # Alle Dateien im 'data' Ordner hochladen oder aktualisieren
    uploaded_ids = uploader.upload_directory(output_dir, recursive=True)
    if uploaded_ids:
        logging.info(f"{len(uploaded_ids)} Dateien wurden erfolgreich hochgeladen/aktualisiert.")
    else:
        logging.warning("Es wurden keine Dateien hochgeladen/aktualisiert.")

if __name__ == "__main__":
    asyncio.run(main())
